In [1]:
import pickle

In [2]:
# WEAR情報取得
with open("related_brands.pkl", "rb") as f:
    related_brands = pickle.load(f)

In [3]:
# Store情報取得
with open("prefectures.pkl", "rb") as f:
    prefectures = pickle.load(f)

In [4]:
# とりあえずマトリクスをつくる
# ひとまずセレクトショップ用
# 命名の正規化はしていないので、ここでlower化したものを比較して、WERAとfasshion pressの違いを吸収しておくのもいいかも（以下だとしてない）

# 構造としてはこんな感じ
# prefectures["北海道"]["北斗"]["stores"]["ABCマート イオン上磯"]["detail"]["取り扱いブランド名"]
brands = {}
for i in prefectures:
    for j in prefectures[i]:
        if len(prefectures[i][j]["stores"]) == 0:
            continue
        for k in prefectures[i][j]["stores"]:
            if len(prefectures[i][j]["stores"][k]["detail"]["取り扱いブランド名"]) == 0:
                continue
            for l in prefectures[i][j]["stores"][k]["detail"]["取り扱いブランド名"]:
                print(l)
                if l in brands.keys():
                    brands[l].extend(prefectures[i][j]["stores"][k]["detail"]["取り扱いブランド名"])
                    brands[l].remove(l)
                else :
                    brands[l] = prefectures[i][j]["stores"][k]["detail"]["取り扱いブランド名"][:]
                    brands[l].remove(l)

RCELO BURLON COUNTY OF MILAN
MARNI
MAISON EUREKA
Maison Margiela
MONCLER
Rick Owens
Y-3
URBAN RESEARCH
EDWIN
BACK NUMBER
Lee
Levi's
Gabardine K.T
K.T KIYOKO TAKASE
COMME ÇA
Gabardine K.T
K.T KIYOKO TAKASE
COMME ÇA
77circa
UN3D.
CLANE
G.V.G.V.
TAN
FLORENT
PONTI
MAISON EUREKA
RIM.ARK
N.HOOLYWOOD
and wander
amp japan
UNUSED
STAMPD
nonnative
FLISTFIA
ROTTWEILER
THE CLOCK HOUSE
spoken words project
Né-net
franche lippee
MARECHAL TERRE
mercibeaucoup,
TOMORROWLAND
UNITED ARROWS
BEAMS
SHIPS
gomme
JUN OKAMOTO
JUNYA WATANABE COMME des GARÇONS
COMME des GARÇONS
ffiXXed
COMME des GARÇONS
PotioR
muller of yoshiokubo
mintdesigns
ISSEY MIYAKE
ASICS
adidas
GRAVIS
CONVERSE
Saucony
Danner
Timberland
NIKE
New Balance
PUMA
Hawkins
Reebok
VANS
ASICS
adidas
GRAVIS
CONVERSE
Saucony
Danner
Timberland
NIKE
New Balance
PUMA
Hawkins
Reebok
VANS
EDWIN
BACK NUMBER
Lee
Levi's
EDWIN
BACK NUMBER
Lee
Levi's
N.HOOLYWOOD
UNDERCOVER
GARNI
CHRISTIAN DADA
UNDERCOVER
G.V.G.V.
soe
DIET BUTCHER
doublet
TOGA
NON TOKYO
MISTERGE

In [5]:
# マトリクス作成
mat_select = {i:{j:0 for j in brands} for i in brands}
for i in brands:
    for j in brands[i]:
        mat_select[i][j] = mat_select[i][j] + 1

In [30]:
# グラフ作成
import networkx as nx
G = nx.Graph()
# 重みありだと動かなかった。（理論見てないから謎のまま）
for i in mat_select:
    for j in mat_select[i]:
        if mat_select[i][j] == 0:
            continue
        G.add_edge(i,j)
        #G.add_weighted_edges_from([(i,j,mat_select[i][j])])

In [34]:
# グラフ描画
# 処理重すぎた
# import matplotlib.pyplot as plt
# import scipy
# nx.draw(G, with_labels = True)
# plt.show()

ModuleNotFoundError: No module named 'scipy'

In [38]:
# スペクトルクラスタリング
from matplotlib import cm
from sklearn.cluster import KMeans
import numpy as np

class graph_clustering():
    def __init__(self, K, n_clusters, lap_kind):
        """
        K : int
        n_clusters : int
        lap_kind : None or str
            Ex. "rw"
        """
        self.lap_kind = lap_kind
        self.K = K
        self.n_clusters = n_clusters

    def _calc_g_laplacian(self, X):
        D = np.diag(np.sum(X, axis=0))
        L = D - X
        self.L = L
        self.D = D
        return L

    def _calc_rw_g_laplacian(self, X):
        L = self._calc_g_laplacian(X)
        Lrw = np.dot(np.linalg.inv(self.D), L)
        self.L = Lrw
        return Lrw

    def _get_K_eigen_vec(self, Lam, V):
        sorted_index = np.argsort(Lam.real)
        Lam_K = Lam[sorted_index][0:self.K]
        V_K = V[sorted_index][0:self.K]

        self.Lam_K = Lam_K
        self.V_K = V_K

        return Lam_K, V_K

    def _Kmeans_V_K(self, V_K):
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=0).fit(V_K.T.real)
        clusters = kmeans.labels_
        self.clusters = clusters
        return clusters

    def laplacian_clustering(self, X):
        """
        X : ndarray
            a matrix representation of undirected graph
        """
        if self.lap_kind is None:
            L = self._calc_g_laplacian(X)
        elif self.lap_kind == "rw":
            L = self._calc_rw_g_laplacian(X)
        else: 
            raise NotImplementedError

        Lam, V = np.linalg.eig(L)
        Lam_K, V_K = self._get_K_eigen_vec(Lam, V)
        clusters = self._Kmeans_V_K(V_K)
        return clusters

# for plot
def get_labels_dic(G):
    """
    G : networkx.classes.graph.Graph
    """
    labels_dic = { key for key in G.nodes.keys() }
    return labels_dic

def get_labels_list(G):
    labels_list = np.array([ key for key in G.nodes.keys() ])
    return labels_list

def split_labels(labels_list, clusters):
    """
    labels_list : list
        return value of get_labels_list function.
    clusters : ndarray
        return value of graph_clustering.laplacian_clustering
    """
    class_uniq = np.unique(clusters)
    node_index_split= []
    labels_split = []
    index = np.arange(clusters.shape[0])
    for class_elem in class_uniq:
        node_index_split.append(list(index[clusters == class_elem]))
        labels_split.append(list(labels_list[clusters == class_elem]))
    return node_index_split, labels_split

def plot_clusters(G, node_index_split):
    """
    G : networkx.classes.graph.Graph
    node_index_split : list (2-dim list)
        return value of split_labels function.
    """
    labels_dic = get_labels_dic(G)
    pos = nx.spring_layout(G)
    colors = [ cm.jet(x) for x in np.arange(0, 1, 1/len(node_index_split)) ]
    for i, nodes in enumerate(node_index_split):
        nx.draw_networkx(G, node_color=colors[i], labels=labels_dic, nodelist=nodes, pos=pos, font_color="r")
    plt.show()

X = np.array(nx.to_numpy_matrix(G))
CLA = int(len(brands) / 10)
GClus = graph_clustering(K=15, n_clusters=CLA, lap_kind="rw")
clusters = GClus.laplacian_clustering(X)
labels_list = get_labels_list(G)
# 酔歩正規化グラフラプラシアン
node_index_split, labels_split = split_labels(labels_list, clusters)
# エラー出るけどよくわからない
plot_clusters(G, node_index_split)

NetworkXError: Node 54 has no position.

In [49]:
# クラスタ番号ごとにブランドを格納
clst_brands = {}
for i in range(1,CLA+1):
    clst_brands[i] = []
    for j in [ind for ind, j in enumerate(clusters) if j == i]:
        clst_brands[i].append(labels_list[j])

In [65]:
for i in clst_brands:
    for j in clst_brands[i]:
        if j == "LAD MUSICIAN":
            print(clst_brands[i])

['DUVETICA', 'Frank & Eileen', 'SUICOKE', 'AALTO', 'Hender Scheme', 'LIMI feu', 'sina suien', 'ts(s)', 'mando', 'PONTI', 'NIKE', 'Cape Heights', 'RIM.ARK', 'SEIKO', 'HAMILTON', 'LAD MUSICIAN', 'THE H.W.DOG&CO.', "Pherrow's", 'RED WING', 'Whitehouse Cox', 'Paul Harnden', 'COOTIE', 'mastermind JAPAN', 'Text', 'CA4LA', 'CAL O LINE', 'K-Swiss', 'SON OF THE CHEESE', 'BLUEBLUE', 'kearny', 'POLYPLOID', 'KAIKO', 'OFF-WHITE c/o VIRGIL ABLOH', 'HELMUT LANG', 'MCM', 'MUSHER', 'Gosha Rubchinskiy', 'evam eva', 'havaianas', 'Bark', 'Patagonia', 'FIRST DOWN', 'Lewis Leathers', 'DSQUARED2', 'Kappa', 'BETONES', 'mikia', 'Marine Serre', 'SELF MADE BY GIANFRANCO VILLEGAS', 'Shinzone']


In [62]:
list(labels_list).index("VANS")

278